In [1]:
#Ethan Vaz Falcao 
import os 
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import seaborn as sns
import numpy as np
%matplotlib inline
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import datasets, linear_model
from scipy.stats import poisson

In [2]:
df_fifa_rank = pd.read_csv("/Users/Ethan Vaz Falcao/Downloads/Data/fifa_ranking-2022-10-06.csv", sep=",")
df_fifa_results = pd.read_csv("/Users/Ethan Vaz Falcao/Downloads/Data/results.csv", sep=",")
#Data sourced from kaggle.com
#https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017?resource=download&select=shootouts.csv

In [3]:
#df_fifa_rank.head()

In [4]:
#df_fifa_results.head()

In [5]:
#Convert the values in the "date" and "rank_data" of the df_fifa_results and df_fifa_rank, to represent date times. 
df_fifa_results["date"] = pd.to_datetime(df_fifa_results["date"])
df_fifa_rank["rank_date"] = pd.to_datetime(df_fifa_rank["rank_date"])
df_fifa_rank = df_fifa_rank.set_index(['rank_date']).groupby(['country_full'], group_keys=False).resample('D').first().fillna(method='ffill').reset_index()
                                    


In [6]:
# Merging the df_fifa_rank and df_fifa_results together 
df = df_fifa_results.merge(df_fifa_rank[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], left_on=["date", "home_team"], right_on=["rank_date", "country_full"])
df = df.merge(df_fifa_rank[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], left_on=["date", "away_team"], right_on=["rank_date", "country_full"], suffixes=("_home", "_away"))

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21483 entries, 0 to 21482
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  21483 non-null  datetime64[ns]
 1   home_team             21483 non-null  object        
 2   away_team             21483 non-null  object        
 3   home_score            21483 non-null  int64         
 4   away_score            21483 non-null  int64         
 5   tournament            21483 non-null  object        
 6   city                  21483 non-null  object        
 7   country               21483 non-null  object        
 8   neutral               21483 non-null  bool          
 9   country_full_home     21483 non-null  object        
 10  total_points_home     21483 non-null  float64       
 11  previous_points_home  21483 non-null  float64       
 12  rank_home             21483 non-null  float64       
 13  rank_change_home

In [8]:
# Dropping the repeated columns from merge 
df= df.drop(["rank_date_home","rank_date_away", "country_full_away","country_full_home",
             "rank_change_away","rank_change_home","country_full_away","country_full_home","previous_points_away","previous_points_home","country","neutral"], axis=1)

In [9]:
#renaming column headers and fixing any contry names
df = df.rename(columns = {'home_score': "home_team_score"})
df = df.rename(columns = {'away_score': "away_team_score"})
df = df.rename(columns = {'rank_home': "home_team_fifa_rank"})
df = df.rename(columns = {'rank_away': "away_team_fifa_rank"})
df = df.rename(columns = {'total_points_home': "home_team_total_fifa_points"})
df = df.rename(columns = {'total_points_away': "away_team_total_fifa_points"})
#df = df.rename(columns = {'neutral': "friendly"})
df = df.rename(columns = {'city': "location"})
df["home_team"] = df["home_team"].str.replace("USA", "United States").str.replace("IR Iran", "Iran").str.replace("Israel", "Palestine")

In [10]:
df = df[(df["date"] >= "2018-8-1")].reset_index(drop=True)

In [11]:
df

,date,home_team,away_team,home_team_score,away_team_score,tournament,location,home_team_total_fifa_points,home_team_fifa_rank,away_team_total_fifa_points,away_team_fifa_rank
0,2018-08-04,Belize,Barbados,1,0,Friendly,San Pedro,1052.00,163.0,1059.00,162.0
1,2018-08-04,Palestine,Iraq,0,3,Friendly,Al-Ram,1449.00,101.0,1534.00,84.0
2,2018-08-15,Guatemala,Cuba,3,0,Friendly,Guatemala City,1263.00,133.0,1018.00,168.0
3,2018-08-18,Andorra,United Arab Emirates,0,0,Friendly,Grödig,1120.00,130.0,1312.00,77.0
4,2018-08-18,Grenada,Jamaica,1,5,Friendly,St. George's,980.00,168.0,1400.00,54.0
...,...,...,...,...,...,...,...,...,...,...,...
3216,2022-09-27,Norway,Serbia,0,2,UEFA Nations League,Oslo,1488.57,36.0,1549.53,25.0
3217,2022-09-27,Sweden,Slovenia,1,1,UEFA Nations League,Stockholm,1563.44,20.0,1372.48,65.0
3218,2022-09-27,Kosovo,Cyprus,5,1,UEFA Nations League,Pristina,1183.90,106.0,1180.52,108.0
3219,2022-09-27,Greece,Northern Ireland,3,1,UEFA Nations League,Athens,1441.45,49.0,1399.10,58.0


In [12]:

# for column_headers in df.columns: 
#     print(column_headers)

In [13]:
# date: date of the match
# home_team: the name of the home team
# away_team: the name of the away team
# home_team_score: full-time home team score
# away_team_score: full-time away team score 
# tournament: the name of the tournament
# location: the name of the city where the match was played
# home_team_total_fifa_points: current total points of home team as of the time of the match
# home_team_fifa_rank: current country rank of home team as of the time of the match
# away_team_total_fifa_points: current total points of away team as of the time of the match
# away_team_fifa_rank: country rank of away team as of the time of the match 


In [14]:
df["goal_difference"] = (
    abs(df["home_team_score"] - df["away_team_score"])
)
df["goal_total"] = (
    abs(df["home_team_score"] + df["away_team_score"])
)



mean_goals = sum(df["home_team_score"])/(len(df))
mean_goals

mean_goals_dF = sum(df["away_team_score"])/(len(df))
mean_goals_dF


1.0698540825830487

In [15]:

df_home = df[['home_team', "home_team_fifa_rank","home_team_score","away_team_score"]]
df_away = df[['away_team', "away_team_fifa_rank","home_team_score","away_team_score"]]

In [16]:
df_home = df_home.rename(columns={'home_team': "Team", "home_team_fifa_rank":"fifa_rank","home_team_score":"Goals","away_team_score":"Goals_Conceded"})
df_away = df_away.rename(columns = {'away_team': "Team", "away_team_fifa_rank":"fifa_rank","home_team_score": "Goals_Conceded","away_team_score":"Goals"})

Calculating Offense Strength is dividing the team’s average number of goals scored by the average number of goals allowed.
Calculating Defence Strength is dividing the team’s average number of goals allowed by the average number of goals allowed.

In [17]:
df_team = pd.concat([df_home,df_away],ignore_index= True).groupby("Team").mean()
df_team["Of_strength"]= df_team["Goals"]/mean_goals
df_team["Df_strength"]= df_team["Goals_Conceded"]/mean_goals_dF

df_team

,fifa_rank,Goals,Goals_Conceded,Of_strength,Df_strength
Team,,,,,
Afghanistan,149.000000,0.888889,1.722222,0.566504,1.609773
Albania,64.476190,1.095238,1.190476,0.698014,1.112746
Algeria,43.106383,2.170213,0.595745,1.383113,0.556847
American Samoa,190.000000,2.000000,7.000000,1.274634,6.542948
Andorra,143.511628,0.511628,1.906977,0.326069,1.782464
...,...,...,...,...,...
Vietnam,97.628571,1.571429,1.028571,1.001498,0.961413
Wales,19.500000,1.125000,1.062500,0.716982,0.993126
Yemen,141.600000,0.300000,2.250000,0.191195,2.103091


In [18]:
df_team["Of_strength"]["Afghanistan"]

0.5665039792463615

In [19]:
#goals expectancy
def goals_prob(home,away):
        goals_exp_home =df_team["Of_strength"][home]*df_team["Df_strength"][away]
        goals_exp_away =df_team["Of_strength"][away]*df_team["Df_strength"][home]
        return goals_exp_home,goals_exp_away
goals_prob("Brazil","Yemen")
# figure out how to add quotes around input


(2.8802950158881506, 0.06844265384288674)

In [20]:
#goals expectancy
def match_score(home,away):
    if home in df_team.index and away in df_team.index:
        exp_home =df_team["Of_strength"][home]*df_team["Df_strength"][away]
        exp_away =df_team["Of_strength"][away]*df_team["Df_strength"][home]
       # return goals_exp_home,goals_exp_away
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, exp_home) * poisson.pmf(y, exp_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        print(points_home, points_away)
        if(points_home>points_away):
            print(home," is likely to win!")
        else:
            print(away," is likely to win!")
    else:
        return (0, 0)

In [21]:
match_score("Morocco","Argentina") 

1.2683887737560724 1.3051999417972162
Argentina  is likely to win!


In [22]:
match_score("Croatia","France")

0.802634495405648 1.951953152829949
France  is likely to win!


In [23]:
match_score("Argentina","Argentina") 

1.294693467373191 1.2946934673731914
Argentina  is likely to win!


In [24]:
tourney = ["Netherlands","Senegal","England","United States","Argentina","Poland","France","Australia","Japan","Spain", "Morocco","Croatia","Brazil","Switerzerland","Portugal","Korea Republic"]


In [25]:
match_score("Morocco","Croatia") 

2.0259468273681938 0.6978671380205097
Morocco  is likely to win!
